In [59]:
import pandas as pd
from utils.load_data import LoadData 

### ground truth

In [60]:
# the ground truth values
load_data = LoadData()
data = load_data.data
data[data['year'] == 2024]

Data lost:  31


,year,month,day,week_day,consumo_max_diario,prod_e,prod_l,Consumo Vivix,MWh/dia,boosting,cor,espessura,extracao_forno,porcentagem_caco,_merge,crossed_threshold
365,2024,1,1,0,6.357671,0.0,0.0,146858.000,6.119117,1.8000,incolor,8.0,801.888266,12.969388,both,0
366,2024,1,2,1,6.353846,0.0,0.0,125399.000,5.224966,1.8000,incolor,6.0,760.639555,13.015363,both,0
367,2024,1,3,2,6.790784,0.0,0.0,142237.000,5.922399,1.8000,incolor,4.0,756.407065,12.955987,both,0
368,2024,1,4,3,7.210391,0.0,0.0,157797.000,6.449902,1.8000,incolor,3.0,751.027689,13.048787,both,0
369,2024,1,5,4,7.028001,1.0,0.0,156202.000,6.508437,1.8000,incolor,3.0,745.614081,13.009411,both,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,2024,3,27,2,8.834712,0.0,0.0,194612.429,8.108851,3.0004,verde,8.0,807.694566,57.992219,both,1
452,2024,3,28,3,8.683421,0.0,0.0,194184.079,8.091003,3.0044,verde,8.0,827.881233,57.979331,both,1
453,2024,3,29,4,9.024435,0.0,1.0,203015.788,8.458991,3.0059,verde,8.0,825.910154,57.996623,both,1
454,2024,3,30,5,9.086995,0.0,1.0,207019.308,8.625804,3.0014,verde,8.0,825.581337,58.019722,both,1


In [61]:
demanda_2024 = data.loc[data['year'] == 2024][['year','month','day','crossed_threshold','consumo_max_diario',]]
demanda_2024['datetime'] = pd.to_datetime(demanda_2024[['year','month','day']])
demanda_2024 = demanda_2024.set_index('datetime')
demanda_2024.drop(columns=['year','month','day'], inplace=True)
demanda_2024.rename(columns={'crossed_threshold':'estouro de demanda', 'consumo_max_diario':'demanda (MWh)'}, inplace=True)
demanda_2024

,estouro de demanda,demanda (MWh)
datetime,,
2024-01-01,0,6.357671
2024-01-02,0,6.353846
2024-01-03,0,6.790784
2024-01-04,0,7.210391
2024-01-05,0,7.028001
...,...,...
2024-03-27,1,8.834712
2024-03-28,1,8.683421
2024-03-29,1,9.024435


Show results function 

In [62]:
def apply_threshold(probabilities, threshold):
    return (probabilities >= threshold).astype(int)

In [63]:
def show_df_results(model_name, new_threshold=None): 
    prediction_data_path = f'results/output_data/{model_name}.csv'
    prediction_data = pd.read_csv(prediction_data_path)
    prediction_data.index = pd.to_datetime(prediction_data['datetime'])
    prediction_data.drop(columns=['datetime'], inplace=True)
    prediction_data = prediction_data[['probabilidade_de_estouro', 'estouro_previsto']]

    print("Prediction data")
    display(prediction_data)

    results_df = pd.merge(prediction_data, demanda_2024, how='inner', left_index=True, right_index=True)

    if new_threshold:
        results_df['estouro_previsto'] = apply_threshold(results_df['probabilidade_de_estouro'], new_threshold)
        print(f"New threshold: {new_threshold}")

    print("Results data")
    display(results_df)
   

    false_negative_error_df =results_df[(results_df['estouro_previsto'] == 0) & (results_df['estouro de demanda'] == 1)]
    print(f'False negative error: {len(false_negative_error_df)} days of {len(results_df[results_df["estouro de demanda"] == 1])} days')
    display(false_negative_error_df)


    false_positive_error_df =results_df[(results_df['estouro_previsto'] == 1) & (results_df['estouro de demanda'] == 0)]
    print(f'False positive error: {len(false_positive_error_df)} days of {len(results_df[results_df["estouro de demanda"] == 0])} days')
    display(false_positive_error_df)

    TP = len(results_df[(results_df["estouro_previsto"] == 1) & (results_df["estouro de demanda"] == 1)]) 
    TN = len(results_df[(results_df["estouro_previsto"] == 0) & (results_df["estouro de demanda"] == 0)])
    FP = len(results_df[(results_df["estouro_previsto"] == 1) & (results_df["estouro de demanda"] == 0)])
    FN = len(results_df[(results_df["estouro_previsto"] == 0) & (results_df["estouro de demanda"] == 1)])

    print(f'Accuracy: {(TP + TN) / (TP + TN + FP + FN)}')
    print(f'Precision: {TP / (TP + FP)}')
    print(f'Recall: {TP / (TP + FN)}')
    print(f'F1 Score: {2 * (TP / (TP + FP) * TP / (TP + FN)) / (TP / (TP + FP) + TP / (TP + FN))}')
    


# Xgboost

In [64]:
show_df_results('xgboost')

Prediction data


,probabilidade_de_estouro,estouro_previsto
datetime,,
2024-01-01,0.005157,0
2024-01-02,0.004496,0
2024-01-03,0.006817,0
2024-01-04,0.004897,0
2024-01-05,0.006186,0
...,...,...
2024-12-27,0.940286,1
2024-12-28,0.905265,1
2024-12-29,0.839121,1


Results data


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-01-01,0.005157,0,0,6.357671
2024-01-02,0.004496,0,0,6.353846
2024-01-03,0.006817,0,0,6.790784
2024-01-04,0.004897,0,0,7.210391
2024-01-05,0.006186,0,0,7.028001
...,...,...,...,...
2024-03-27,0.980687,1,1,8.834712
2024-03-28,0.977460,1,1,8.683421
2024-03-29,0.986550,1,1,9.024435


False negative error: 7 days of 14 days


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-03-11,0.026346,0,1,8.802856
2024-03-13,0.205438,0,1,8.534767
2024-03-14,0.092298,0,1,8.879012
2024-03-15,0.092298,0,1,8.761183
2024-03-24,0.203780,0,1,8.745083
2024-03-25,0.007916,0,1,8.676560
2024-03-26,0.009029,0,1,9.144856


False positive error: 0 days of 77 days


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,


Accuracy: 0.9230769230769231
Precision: 1.0
Recall: 0.5
F1 Score: 0.6666666666666666


Xgboost v2

In [88]:
# testing xgboost to compare with the best previous model
show_df_results('xgboost', new_threshold=0.08)

Prediction data


,probabilidade_de_estouro,estouro_previsto
datetime,,
2024-01-01,0.005157,0
2024-01-02,0.004496,0
2024-01-03,0.006817,0
2024-01-04,0.004897,0
2024-01-05,0.006186,0
...,...,...
2024-12-27,0.940286,1
2024-12-28,0.905265,1
2024-12-29,0.839121,1


New threshold: 0.08
Results data


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-01-01,0.005157,0,0,6.357671
2024-01-02,0.004496,0,0,6.353846
2024-01-03,0.006817,0,0,6.790784
2024-01-04,0.004897,0,0,7.210391
2024-01-05,0.006186,0,0,7.028001
...,...,...,...,...
2024-03-27,0.980687,1,1,8.834712
2024-03-28,0.977460,1,1,8.683421
2024-03-29,0.986550,1,1,9.024435


False negative error: 3 days of 14 days


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-03-11,0.026346,0,1,8.802856
2024-03-25,0.007916,0,1,8.676560
2024-03-26,0.009029,0,1,9.144856


False positive error: 2 days of 77 days


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-02-07,0.091901,1,0,7.356156
2024-03-19,0.092832,1,0,7.774334


Accuracy: 0.945054945054945
Precision: 0.8461538461538461
Recall: 0.7857142857142857
F1 Score: 0.8148148148148149


### Random Forest

In [66]:
show_df_results('random_forest')

Prediction data


,probabilidade_de_estouro,estouro_previsto
datetime,,
2024-01-01,0.008,0
2024-01-02,0.010,0
2024-01-03,0.004,0
2024-01-04,0.004,0
2024-01-05,0.022,0
...,...,...
2024-12-27,0.138,0
2024-12-28,0.098,0
2024-12-29,0.112,0


Results data


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-01-01,0.008,0,0,6.357671
2024-01-02,0.010,0,0,6.353846
2024-01-03,0.004,0,0,6.790784
2024-01-04,0.004,0,0,7.210391
2024-01-05,0.022,0,0,7.028001
...,...,...,...,...
2024-03-27,0.732,1,1,8.834712
2024-03-28,0.622,1,1,8.683421
2024-03-29,0.814,1,1,9.024435


False negative error: 6 days of 14 days


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-03-11,0.296,0,1,8.802856
2024-03-14,0.200,0,1,8.879012
2024-03-15,0.396,0,1,8.761183
2024-03-24,0.198,0,1,8.745083
2024-03-25,0.122,0,1,8.676560
2024-03-26,0.230,0,1,9.144856


False positive error: 0 days of 77 days


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,


Accuracy: 0.9340659340659341
Precision: 1.0
Recall: 0.5714285714285714
F1 Score: 0.7272727272727273


Random Forest v2

In [77]:
show_df_results('random_forest', new_threshold=0.1)

Prediction data


,probabilidade_de_estouro,estouro_previsto
datetime,,
2024-01-01,0.008,0
2024-01-02,0.010,0
2024-01-03,0.004,0
2024-01-04,0.004,0
2024-01-05,0.022,0
...,...,...
2024-12-27,0.138,0
2024-12-28,0.098,0
2024-12-29,0.112,0


New threshold: 0.1
Results data


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-01-01,0.008,0,0,6.357671
2024-01-02,0.010,0,0,6.353846
2024-01-03,0.004,0,0,6.790784
2024-01-04,0.004,0,0,7.210391
2024-01-05,0.022,0,0,7.028001
...,...,...,...,...
2024-03-27,0.732,1,1,8.834712
2024-03-28,0.622,1,1,8.683421
2024-03-29,0.814,1,1,9.024435


False negative error: 0 days of 14 days


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,


False positive error: 1 days of 77 days


,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-03-19,0.304,1,0,7.774334


Accuracy: 0.989010989010989
Precision: 0.9333333333333333
Recall: 1.0
F1 Score: 0.9655172413793104


KNN